In [2]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

##### 1 task

In [4]:
from numpy.linalg import matrix_power

def beta_cov(x, y):
    n = x.shape[0]
    beta_star = np.matmul(matrix_power((1/n)*np.matmul(x.T, x), -1),  (1/n)*np.matmul(x.T, y))
    H = (1/n)*np.matmul(x.T, x)
    # print(beta_star.shape)
    # print(x.shape)
    # print(y.shape)
    # print(x.T.shape)
    # print((y - np.matmul(x, beta_star)).shape)
    D = np.diag((y - np.matmul(x, beta_star))[:,0])**2
    # print(D.shape)
    V = np.matmul(x.T, np.matmul(D, x))/n
    # print("V:", V.shape)
    # print(H.shape)
    H_ = matrix_power(H, -1)
    sigma = np.matmul(H_, np.matmul(V, H_))
    return beta_star, sigma


rng = np.random.default_rng(1337)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps
x.shape[0]

print(*beta_cov(x, y), sep='\n')

[[-1.0878702 ]
 [-0.5688717 ]
 [ 0.93436482]]
[[0.67038035 0.0662398  0.05856229]
 [0.0662398  0.60565119 0.0659719 ]
 [0.05856229 0.0659719  1.91253231]]


##### 2 task

In [65]:
from numpy.linalg import matrix_power

def beta_cov(x, y):
    n = x.shape[0]
    beta_star = np.matmul(matrix_power(
        (1/n)*np.matmul(x.T, x), -1),  (1/n)*np.matmul(x.T, y))
    H = (1/n)*np.matmul(x.T, x)
    D = np.diag((y - np.matmul(x, beta_star))[:, 0])**2
    V = np.matmul(x.T, np.matmul(D, x))/n
    H_ = matrix_power(H, -1)
    sigma = np.matmul(H_, np.matmul(V, H_))
    return beta_star, sigma


def interval(x, y, c):
    beta_star, sigma_star = beta_cov(x, y)
    sigma_square = np.matmul(c, np.matmul(sigma_star, c.T))
    # print(sigma_square.shape)
    ci = ss.norm.interval(0.95, loc=0, scale = sigma_square)/np.sqrt(len(y))
    ci = ci.reshape((1,-1))[0]
    conf_interval = c@beta_star + ci
    return conf_interval[0]


data = pd.read_csv(
    'https://stepik.org/media/attachments/lesson/832665/Fish.csv')
data = data.drop(40, axis=0)

x = data[['Length1', 'Height', 'Width']].apply(np.log)
x['const'] = 1
y = data[['Weight']].apply(np.log)
c = np.array([[1, 1, 1, 0]])  # в таком порядке, так как константа последняя

print(interval(x.values, y.values, c))

[2.99449402 3.02061303]


##### 3 task

In [84]:
def beta_WLS(x, y, w):
    W = np.diag(w(x))
    beta = np.linalg.solve(np.dot(np.dot(x.T, W), x), np.dot(np.dot(x.T, W), y))
    return beta


rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps * np.abs(x[:, [0]])
def w(x): return np.abs(x[:, 0])


print(beta_WLS(x, y, w))

[[-0.19503921]
 [ 0.5637669 ]
 [-2.00190378]]


##### 4 task

In [95]:
def beta_WLS(x, y, w):
    W = np.diag(w(x))
    beta = np.linalg.solve(np.dot(np.dot(x.T, W), x), np.dot(np.dot(x.T, W), y))
    return beta

def cov_WLS(x, y, w):
    beta_star = beta_WLS(x, y, w)
    D = np.diag((y - np.matmul(x, beta_star))[:,0])**2
    V = np.matmul(x.T, np.matmul(D, x))/x.shape[0]
    print(V.shape)
    # W = np.diag(w(x))
    # H = np.matmul(x.T, W)
    # H = np.matmul(np.matmul(x.T, np.matmul(W, x)), H)
    # H = np.matmul(x, H) # 100 x 100
    # H_ = matrix_power(H, -1)
    # sigma = np.matmul(H_, np.matmul(V, H_))
    # return sigma

rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps * np.abs(x[:, [0]])
def w(x): return np.abs(x[:, 0])


print(cov_WLS(x, y, w))

(3, 3)
None


##### 5 task

##### 6 task

In [103]:
def psi(x, y, beta):
    mu = np.exp(np.matmul(x, beta))
    psi_t = (y - mu) * x
    return psi_t.T


rng = np.random.default_rng(0)
x = ss.norm.rvs(size=(1, 5), random_state=rng)
beta = ss.norm.rvs(size=(5, 1), random_state=rng)
y = ss.poisson(np.exp(x @ beta)).rvs(random_state=rng)

print(psi(x, y, beta))

[[-0.24589836]
 [ 0.25836564]
 [-1.25251412]
 [-0.20515964]
 [ 1.04764166]]
